# USGS Earthquake Data with pystac-monty

This notebook demonstrates how to use pystac-monty to process USGS earthquake data and visualize it using interactive maps. We'll:

1. Fetch the latest major earthquakes from USGS
2. Convert them to STAC items using pystac-monty
3. Display events on an interactive map
4. Allow selection of events to view related hazards and impacts
5. Explore the Monty STAC model and its metadata

Let's begin by importing the necessary libraries.

In [ ]:
# Basic libraries
import json
from datetime import datetime, timedelta

# Visualization libraries
import pandas as pd

# STAC and pystac-monty
import pytz
import requests

from pystac_monty.extension import MontyExtension
from pystac_monty.geocoding import WorldAdministrativeBoundariesGeocoder
from pystac_monty.sources.usgs import USGSDataSource, USGSTransformer

# Import STAC helper functions
import sys
sys.path.append('..')
from stac_helpers import (
    check_stac_api_availability,
    check_collection_exists,
    create_collection_from_file,
    create_collection_fallback,
    add_items_to_collection
)

ModuleNotFoundError: No module named 'folium'

## 1. Fetch Recent Major Earthquakes from USGS

Let's fetch earthquakes with magnitude 5.5+ from the past 10 days using the USGS API.

In [ ]:
# Define USGS API endpoint for fetching earthquake data
def fetch_usgs_earthquakes(min_magnitude=5.5, days=60):
    """
    Fetch earthquake data from USGS API

    Parameters:
    - min_magnitude: Minimum magnitude to filter earthquakes (default: 4.5)
    - days: Number of days to look back (default: 30)

    Returns:
    - List of earthquake data as dictionaries
    """
    # Calculate the start time (days ago from now)
    start_time = datetime.now() - timedelta(days=days)
    start_time_str = start_time.strftime("%Y-%m-%dT%H:%M:%S")

    # USGS earthquake API endpoint
    url = "https://earthquake.usgs.gov/fdsnws/event/1/query"

    # Parameters for the API request
    params = {"format": "geojson", "starttime": start_time_str, "minmagnitude": min_magnitude, "orderby": "time"}

    # Make the request to the USGS API
    response = requests.get(url, params=params)
    data = response.json()

    print(f"Found {len(data['features'])} earthquakes with magnitude {min_magnitude}+ in the last {days} days")
    return data




# Fetch recent major earthquakes
earthquake_data = fetch_usgs_earthquakes(min_magnitude=5.5, days=60)
earthquake_features = earthquake_data["features"]

In [ ]:
# Extract relevant information into a DataFrame
earthquakes_df = pd.DataFrame(
    [
        {
            "id": eq["id"],
            "title": eq["properties"]["title"],
            "time": datetime.fromtimestamp(eq["properties"]["time"] / 1000, pytz.UTC),
            "magnitude": eq["properties"]["mag"],
            "place": eq["properties"]["place"],
            "longitude": eq["geometry"]["coordinates"][0],
            "latitude": eq["geometry"]["coordinates"][1],
            "depth": eq["geometry"]["coordinates"][2],
            "tsunami": bool(eq["properties"].get("tsunami")),
            "felt": eq["properties"].get("felt") or 0,
        }
        for eq in earthquake_features
    ]
)

# Sort by magnitude (descending)
earthquakes_df = earthquakes_df.sort_values(by="magnitude", ascending=False)

# Display the DataFrame
earthquakes_df

## 2. Creating Synthetic STAC Items

Let's create synthetic STAC items from the earthquake data using the pystac-monty extension.

In [ ]:
# Initialize the geocoder
geocoder = WorldAdministrativeBoundariesGeocoder("../tests/data-files/world-administrative-boundaries.fgb")

# Create a list to store all synthetic STAC items
all_stac_items = []

# Loop through the earthquake data and create synthetic STAC items
for earthquake in earthquakes_df.to_dict(orient="records"):
    # Define the USGS API endpoint for fetching event detailed geojson
    event_url = f"https://earthquake.usgs.gov/earthquakes/feed/v1.0/detail/{earthquake['id']}.geojson"

    # Get the event data
    event_data = requests.get(event_url).text
    event_data_json = json.loads(event_data)

    # Find the losses data in the event data
    try:
        losses_data_url = event_data_json["properties"]["products"]["losspager"][0]["contents"]["json/losses.json"]["url"]
        losses_data = requests.get(losses_data_url).text
    except KeyError:
        losses_data = None

    # Setup the transformer
    data_source = USGSDataSource(event_url, event_data, losses_data)
    transformer = USGSTransformer(data_source, geocoder)

    # Create the synthetic STAC item
    items = transformer.make_items()
    all_stac_items.extend(items)

# Create synthetic STAC items
print(f"Created {len(all_stac_items)} STAC items from {len(earthquake_features)} earthquakes")

In [ ]:
# Separate the STAC items by role
event_items = []
hazard_items = []
impact_items = []

for item in all_stac_items:
    roles = item.properties.get("roles", [])
    if "event" in roles:
        event_items.append(item)
    elif "hazard" in roles:
        hazard_items.append(item)
    elif "impact" in roles:
        impact_items.append(item)

print(f"Events: {len(event_items)}, Hazards: {len(hazard_items)}, Impacts: {len(impact_items)}")

## 3. Loading STAC Items into a STAC API using the Transaction API

Now that we have created STAC items from the USGS earthquake data, let's load them into a STAC API using the transaction API. The transaction API allows us to create, update, and delete STAC items in a STAC API.

We'll use the predefined collections from the monty-stac-extension examples folder:
- usgs-events: For event items
- usgs-hazards: For hazard items
- usgs-impacts: For impact items

If these collections don't exist in the STAC API, we'll create them based on the predefined collection definitions.

In [ ]:
# Import necessary libraries for STAC API interaction
import requests
import json
import pystac
import os

In [ ]:
# Define the STAC API endpoint
# Replace with your actual STAC API endpoint
stac_api_url = "https://montandon-eoapi-stage.ifrc.org/stac"

# Define the collection IDs for each type of item
# These match the predefined collections in monty-stac-extension/examples
event_collection_id = "usgs-events"
hazard_collection_id = "usgs-hazards"
impact_collection_id = "usgs-impacts"

# Define paths to the predefined collection definitions
event_collection_path = "../monty-stac-extension/examples/usgs-events/usgs-events.json"
hazard_collection_path = "../monty-stac-extension/examples/usgs-hazards/usgs-hazards.json"
impact_collection_path = "../monty-stac-extension/examples/usgs-impacts/usgs-impacts.json"

# Check if the STAC API is available
api_available = check_stac_api_availability(stac_api_url)

In [ ]:
# Check if the collections exist and create them if they don't
if api_available:
    # Check and create event collection if needed
    event_collection_exists = check_collection_exists(stac_api_url, event_collection_id)
    if not event_collection_exists:
        print(f"\nAttempting to create collection '{event_collection_id}'...")
        event_collection_created = create_collection_from_file(stac_api_url, event_collection_path)
        if not event_collection_created:
            print("Trying fallback method to create event collection...")
            event_collection_created = create_collection_fallback(
                stac_api_url, 
                event_collection_id, 
                "USGS Earthquake events processed with pystac-monty",
                ["event", "source"]
            )
        event_collection_exists = event_collection_created
    
    # Check and create hazard collection if needed
    hazard_collection_exists = check_collection_exists(stac_api_url, hazard_collection_id)
    if not hazard_collection_exists:
        print(f"\nAttempting to create collection '{hazard_collection_id}'...")
        hazard_collection_created = create_collection_from_file(stac_api_url, hazard_collection_path)
        if not hazard_collection_created:
            print("Trying fallback method to create hazard collection...")
            hazard_collection_created = create_collection_fallback(
                stac_api_url, 
                hazard_collection_id, 
                "USGS Earthquake hazards processed with pystac-monty",
                ["hazard", "source"]
            )
        hazard_collection_exists = hazard_collection_created
    
    # Check and create impact collection if needed
    impact_collection_exists = check_collection_exists(stac_api_url, impact_collection_id)
    if not impact_collection_exists:
        print(f"\nAttempting to create collection '{impact_collection_id}'...")
        impact_collection_created = create_collection_from_file(stac_api_url, impact_collection_path)
        if not impact_collection_created:
            print("Trying fallback method to create impact collection...")
            impact_collection_created = create_collection_fallback(
                stac_api_url, 
                impact_collection_id, 
                "USGS Earthquake impacts processed with pystac-monty",
                ["impact", "source"]
            )
        impact_collection_exists = impact_collection_created
    
    if not (event_collection_exists and hazard_collection_exists and impact_collection_exists):
        print("\nWarning: One or more collections could not be created in the STAC API.")
        print("Some items may not be added to the STAC API.")
else:
    print("STAC API is not available. Skipping collection checks and creation.")

In [ ]:
# Add the items to their respective collections if the API is available
if api_available:
    if event_collection_exists:
        print("Adding event items to the collection...")
        event_success, event_failed = add_items_to_collection(stac_api_url, event_collection_id, event_items)
    else:
        print("Skipping adding event items because the collection doesn't exist")
        event_success, event_failed = 0, len(event_items)
    
    if hazard_collection_exists:
        print("\nAdding hazard items to the collection...")
        hazard_success, hazard_failed = add_items_to_collection(stac_api_url, hazard_collection_id, hazard_items)
    else:
        print("Skipping adding hazard items because the collection doesn't exist")
        hazard_success, hazard_failed = 0, len(hazard_items)
    
    if impact_collection_exists:
        print("\nAdding impact items to the collection...")
        impact_success, impact_failed = add_items_to_collection(stac_api_url, impact_collection_id, impact_items)
    else:
        print("Skipping adding impact items because the collection doesn't exist")
        impact_success, impact_failed = 0, len(impact_items)
    
    total_success = event_success + hazard_success + impact_success
    total_failed = event_failed + hazard_failed + impact_failed
    
    print(f"\nSummary: Added {total_success} items successfully, {total_failed} items failed")
else:
    print("Skipping adding items to collections because the API is not available")